In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pylab as plt
import scipy.special
import scipy.optimize
import itertools
import pickle
import imp

import cvpinns
imp.reload(cvpinns)

tf.keras.backend.set_floatx("float64")

In [2]:
#analytical solution, 1 rarefaction, 3 shock
def sod(xt,Ul,Ur,gamma=1.4):
    rhol,jl,El = Ul
    rhor,jr,Er = Ur
    
    ul = jl/rhol
    ur = jr/rhor
    pl = (gamma-1.)*(El-.5*rhol*ul**2)
    pr = (gamma-1.)*(Er-.5*rhor*ur**2)
    
    cl = np.sqrt(gamma*pl/rhol)
    cr = np.sqrt(gamma*pr/rhor)

    def phil(p):
        if p<=pl:
            u = ul + 2*cl/(gamma-1) * (1.-(p/pl)**((gamma-1)/(2*gamma)))
        else:
            u = ul + 2*cl/np.sqrt(2*gamma*(gamma-1)) * (1.-p/pl)/np.sqrt(1+p/pl * (gamma+1)/(gamma-1) )
        return u

    def phir(p):
        if p<=pr:
            u = ur - 2*cr/(gamma-1) * (1.-(p/pr)**((gamma-1)/(2*gamma)))
        else:
            u = ur - 2*cr/np.sqrt(2*gamma*(gamma-1)) * (1.-p/pr)/np.sqrt(1+p/pr * (gamma+1)/(gamma-1) )
        return u

    def phi(p):
        return phir(p) - phil(p)

    ps = scipy.optimize.root_scalar(phi,x0=pr,x1=pl).root
    us = phil(ps)
    rhosr = (1 + (gamma+1)/(gamma-1) * ps/pr)/(ps/pr + (gamma+1)/(gamma-1)) * rhor
    rhosl = (ps/pl)**(1./gamma)*rhol
    
    s = (rhosr * us - rhor*ur)/(rhosr - rhor)

    csl = np.sqrt(gamma*ps/rhosl)
    laml = ul - cl
    lamsl = us - csl
    xtr = np.reshape(xt,(np.prod(xt.shape[0:-1]),2))

    u = np.zeros((xtr.shape[0],3))
    
    for i,xti in enumerate(xtr):
        if xti[0]<1e-12:
            if xti[1]<0:
                rho = rhol
                v = ul
                p = pl
            else:
                rho = rhor
                v = ur
                p = pr
        else:                
            xi = xti[1]/xti[0]
            if xi <= laml:
                rho = rhol
                v = ul
                p = pl
            elif xi > laml and xi <= lamsl:
                v = ((gamma-1)*ul + 2*(cl+xi))/(gamma+1)
                rho = (rhol**gamma*(v-xi)**2/(gamma*pl))**(1./(gamma-1))
                p = (pl/rhol**gamma)*rho**gamma
            elif xi > lamsl and xi <= us:
                rho = rhosl
                v = us
                p = ps
            elif xi > us and xi <= s:
                rho = rhosr
                v = us
                p = ps
            else:
                rho = rhor
                v = ur
                p = pr
        
        u[i,0] = rho
        u[i,1] = rho*v
        u[i,2] = p/(gamma-1) + .5*rho*v**2
    return np.reshape(u,xt.shape[0:-1]+(3,))

In [3]:
#domain
L = [[0,1],[-2.5,2.5]]

#left and right states
Ul = [3.,0,3./.4]
Ur = [1.,0.,1./.4]

#compute analytical solution
x_data = np.transpose(np.mgrid[L[0][0]:L[0][1]:100j,L[1][0]:L[1][1]:100j],(1,2,0))
u_data = sod(x_data,Ul,Ur)

In [4]:
#grid size
nx =[64,64]

#quadrature specification
pts0=2
pts1=2
xi0,wi0 = scipy.special.roots_legendre(pts0)
xi1,wi1 = scipy.special.roots_legendre(pts1)
wi01 = np.outer(wi0,wi1)
xi01 = np.transpose(np.meshgrid(xi0,xi1,indexing='ij'),(1,2,0))
quad = {'0':(xi0,wi0),'1':(xi1,wi1),'01':(xi01,wi01)}




In [5]:
#neural network for solution
width = 64
depth = 8
dimi = 2
dimo = 3
act = tf.nn.relu
u = tf.keras.Sequential(
                 [tf.keras.layers.Dense(width,activation=act,input_shape=(None,dimi),dtype=tf.float64)]
                +[tf.keras.layers.Dense(width,activation=act,dtype=tf.float64) for _ in range(depth-1)]
                +[tf.keras.layers.Dense(dimo,dtype=tf.float64,use_bias=False)]
               )

#neural network for EOS
width = 4
depth = 4
dimi = 2
dimo = 1
act = tf.nn.tanh
EOS = tf.keras.Sequential(
                 [tf.keras.layers.Dense(width,activation=act,input_shape=(None,dimi),dtype=tf.float64)]
                +[tf.keras.layers.Dense(width,activation=act,dtype=tf.float64) for _ in range(depth-1)]
                +[tf.keras.layers.Dense(dimo,dtype=tf.float64,use_bias=False)]
                )

In [6]:
#PDE for euler equations
F0_I = u

F0_H = u

@tf.function
def F0_L(x0):
    return tf.expand_dims(tf.cast(x0[...,1]< 0,tf.float64),-1)*Ul \
          +tf.expand_dims(tf.cast(x0[...,1]>=0,tf.float64),-1)*Ur





@tf.function
def eulerFlux(ux1):
    p = .4*(ux1[...,2] - .5*tf.math.divide_no_nan(ux1[...,1]**2,ux1[...,0]))
    return tf.stack([
                        ux1[...,1],
                        tf.math.divide_no_nan(ux1[...,1]**2,ux1[...,0]) + p,
                        (ux1[...,2] + p) * tf.math.divide_no_nan(ux1[...,1],ux1[...,0])
                ],-1)

@tf.function
def F1_I(x1):
    return eulerFlux(u(x1))

@tf.function
def F1_L(x1):
    return eulerFlux(tf.ones(x1.shape[0:-1]+[1],tf.float64)*Ul)

@tf.function
def F1_H(x1):
    return eulerFlux(tf.ones(x1.shape[0:-1]+[1],tf.float64)*Ur)


@tf.function
def F01(x01):
    return tf.zeros(x01.shape[0:-1]+[3],tf.float64)

F = {'0I':F0_I,'0L':F0_L,'0H':F0_H,
     '1I':F1_I,'1L':F1_L,'1H':F1_H,
     '01':F01}

euler = cvpinns.PDE(L,nx,quad,F)

In [7]:
#PDIE for entropy inequality
@tf.function
def getsp(uxi):
    rhoe = tf.stack([uxi[...,0],tf.math.divide_no_nan(uxi[...,2],uxi[...,0]) - 0.5*tf.math.divide_no_nan(uxi[...,1],uxi[...,0])**2],-1)
    with tf.GradientTape() as g:
        g.watch(rhoe)
        s = EOS(rhoe)
        ss = tf.reduce_sum(s)

    J = g.gradient(ss,rhoe)
    p = tf.expand_dims(-rhoe[...,0]**2*tf.math.divide_no_nan(J[...,0],J[...,1]),-1)
    return s,p

@tf.function
def Q0_I(x0):
    ux0 = u(x0)
    s,p = getsp(ux0)
    return -tf.expand_dims(ux0[...,0],-1)*s

Q0_H = Q0_I

@tf.function
def Q0_L(x0):
    ux0 = F0_L(x0)
    s,p = getsp(ux0)
    return -tf.expand_dims(ux0[...,0],-1)*s



@tf.function
def Q1_I(x1):
    ux1 = F0_L(x1)
    s,p = getsp(ux1)
    return -tf.expand_dims(ux1[...,1],-1)*s

@tf.function
def Q1_L(x1):
    ux1 = tf.ones(x1.shape[0:-1]+[1],tf.float64)*Ul
    s,p = getsp(ux1)
    return -tf.expand_dims(ux1[...,1],-1)*s

@tf.function
def Q1_H(x1):
    ux1 = tf.ones(x1.shape[0:-1]+[1],tf.float64)*Ur
    s,p = getsp(ux1)
    return -tf.expand_dims(ux1[...,1],-1)*s

@tf.function
def Q01(x01):
    return tf.zeros(x01.shape[0:-1]+[1],tf.float64)

Q = {'0I':Q0_I,'0L':Q0_L,'0H':Q0_H,
     '1I':Q1_I,'1L':Q1_L,'1H':Q1_H,
     '01':Q01}

euler_ent = cvpinns.PDE(L,nx,quad,Q)

In [8]:
@tf.function
def getLoss():
    return (
                tf.reduce_sum(euler.getRES()**2)                     #PDE residual
              + tf.reduce_sum(tf.nn.relu(euler_ent.getRES())**2)     #entropy inequality residual
            )

In [ ]:
#Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

#optimize loss over u and EOS
varlist = u.variables

#training function
@tf.function
def GD():
    with tf.GradientTape() as g:
        loss = getLoss()
    G = g.gradient(loss,varlist)
    opt.apply_gradients(zip(G,varlist))


for i in range(20000):
    GD()
    if not(i%100):
        print('Loss:',getLoss().numpy())

Loss: 14.305805164165939
Loss: 0.09231756825818897
Loss: 0.030609281211821264
Loss: 0.01195495404346707
Loss: 0.004530937819321031


In [ ]:
fig,ax = plt.subplots(3,figsize=(6,10),sharex=True)
ax[0].plot(euler.x[-1,:,1],u(euler.x)[-1,:,0])
ax[1].plot(euler.x[-1,:,1],u(euler.x)[-1,:,1])
ax[2].plot(euler.x[-1,:,1],u(euler.x)[-1,:,2])